# Quantum Credit Risk Analysis

*Authors: Arjun Puppala, Jose Ramon Aleman, Marti Ciurana*

*Tutor: Giulio Gasbarri*

## Abstract

We introduce and examine a quantum algorithm for estimating credit risk that outperforms Monte Carlo simulations on traditional computers. We calculate the economic capital required, which is the difference between the Value at Risk (VaR) and the predicted loss distribution value (LDV). The calculation of the economic capital required is of interest, because it quantifies the amount of capital required to be solvent at a certain level of confidence, making it an important risk statistic. We implement this problem for a realistic loss distribution and analyze its scaling to a realistic problem size. We present estimates of the overall number of necessary qubits, the expected circuit depth, and how this translates into an expected runtime on future fault-tolerant quantum hardware under acceptable assumptions.


## Introduction

Credit risk analysis is a form of analysis performed by a credit analyst to determine a borrower’s ability to meet their debt obligations. The purpose of credit analysis is to determine the creditworthiness of borrowers by quantifying the risk of loss that the lender is exposed to. The three factors that lenders use to quantify credit risk include the probability of default, loss given default, and exposure at default. [1]

The use of quantum computing could allow faster and accurate solutions, for example determining a credit risk profile of a client. It is said that financial institutions that can make the most of quantum computing are likely to see significant benefits. Banks and asset managers optimize portfolios based on computationally intense models that process large sets of variables. For this reason, they will be able to more effectively analyze large or unstructured data sets.

In valuation, for example, the ability to speedily identify an optimal risk-adjusted portfolio is likely to create significant competitive advantage. For loan and bond portfolios, more precise estimates of credit exposures should lead to better optimization decisions.

American and European banks are also exploring quantum computing opportunities. [2]

- Bank of America strategist said quantum computing would be “as revolutionary in the 2020s as smartphones were in the 2010s.”
- BBVA has formed a partnership to explore portfolio optimization and more efficient Monte Carlo modelling.
- Caixa Bank is running a trial hybrid framework of quantum and conventional computing with the aim of better classifying credit risk profiles.

## Description of the Problem

Default risk is the risk that a lender takes on in the chance that a borrower will be unable to make the required payments on their debt obligation. 

Whenever a lender extends credit to a borrower, there is a chance that the loan amount will not be paid back. The measurement that looks at this probability is the default risk. Default risk does not only apply to individuals who borrow money, but also to companies that issue bonds and due to financial constraints, are not able to make interest payments on those bonds. Whenever a lender extends credit, calculating the default risk of a borrower is crucial as part of its risk management strategy. Whenever an investor is evaluating an investment, determining the financial health of a company or private person is crucial in gauging investment risk.

Lenders generally examine a company's financial statements and employ several financial ratios to determine the likelihood of debt repayment. Free cash flow is the cash that is generated after the company reinvests in itself and is calculated by subtracting capital expenditures from operating cash flow. Free cash flow is used for things such as debt and dividend payments. A free cash flow figure that is near zero or negative indicates that the company may be having trouble generating the cash necessary to deliver on promised payments. This could indicate a higher default risk.

The risk of default on a debt arises from a borrower failing to make the required payments, for example:

- A consumer may fail to make a payment due on a mortgage loan, credit card, line of credit, or other loan.
- A company is unable to repay asset-secured fixed or floating charge debt.
- A business or consumer does not pay a trade invoice when due

### Problem Definition

We want to analyze the credit risk of a portfolio of $K$ assets. The default probability of every asset $k$ follows a Gaussian Conditional Independence model, i.e., given a value $z$ sampled from a latent random variable $Z$ following a standard normal distribution, the default probability of asset $k$ is given by:

$ p_k(z) = F\left( \frac{F^{-1}(p_k^0) - \sqrt{\rho_k}z}{\sqrt{1 - \rho_k}} \right) $

 
where $F$ denotes the cumulative distribution function of $Z$, $p_k^0$ is the default probability of asset $k$ for $z=0$ and $\rho_k$ is the sensitivity of the default probability of asset $k$ with respect to $Z$. Thus, given a concrete realization of $Z$ the individual default events are assumed to be independent from each other.

We are interested in analyzing risk measures of the total loss

$ L = \sum_{k=1}^K \lambda_k X_k(Z) $

where $\lambda_k$ denotes the loss given default of asset $k$, and given $Z$, $X_k(Z)$ denotes a Bernoulli variable representing the default event of asset $k$. More precisely, we are interested in the expected value $\mathbb{E}[L]$, the Value at Risk (VaR) of $L$ and the Conditional Value at Risk of $L$ (also called Expected Shortfall).

Where VaR and CVaR are defined as

$ \text{VaR}_{\alpha}(L) = \inf \{ x \mid \mathbb{P}[L <= x] \geq 1 - \alpha \} $

with confidence level $\alpha \in [0, 1]$, and

$\text{CVaR}_{\alpha}(L) = \mathbb{E}[ L \mid L \geq \text{VaR}_{\alpha}(L) ]$


### Problem Parameters

The problem is defined by the following parameters:

1. Number of qubits used to represent $Z$, denoted by $n_z$
2. Truncation value for $Z$, denoted by $z_\text{max}$, i.e., $Z$ is assumed to take $2^{n_z}$ equidistant values in $\{-z_\text{max},...,+z_\text{max}\}$
3. Base default probabilities for each asset $p_0^k \in (0,1)$, $k=1, ..., K$  
4. Sensitivities of the default probabilities with respect to $Z$, denoted by $\rho_k \in [0,1)$ 
5. Loss given default for asset $k$, denoted by $\lambda_k$
6. confidence level for VaR / CVaR $\alpha \in [0,1]$.

In [ ]:
n_z = 2
z_max = 2
z_values = np.linspace(-z_max, z_max, 2 ** n_z)
p_zeros = [0.15, 0.25]
rhos = [0.1, 0.05]
lgd = [1, 2]
K = len(p_zeros)
alpha = 0.05

## Methods

### How to Estimate the Risk

Value at risk (VaR) is a statistic that quantifies the extent of possible financial losses within a firm, portfolio, or position over a specific time frame. 

A financial firm, for example, may determine an asset has a 3% one-month VaR of 2%, representing a 3% chance of the asset declining in value by 2% during the one-month time frame. The conversion of the 3% chance of occurrence to a daily ratio places the odds of a 2% loss at one day per month.

The CVar quantifies the average expected loss.

### Montecarlo

The Monte Carlo method is a broad class of computational algorithms that rely on repeated random sampling to obtain numerical results. The underlying concept is to use randomness to solve problems that might be deterministic in principle. Monte Carlo Simulation predicts a set of outcomes based on an estimated range of values versus a set of fixed input values. 

In other words, a Monte Carlo Simulation builds a model of possible results by leveraging a probability distribution, such as a uniform or normal distribution, for any variable that has inherent uncertainty. It, then, recalculates the results over and over, each time using a different set of random numbers between the minimum and maximum values. In a typical Monte Carlo experiment, this exercise can be repeated thousands of times to produce a large number of likely outcomes.

Several combinations of different scenarios are tested and the result of each scenario is analyzed. Therefore it’s slow and expensive.

### Grover's Algorithm

In this section, we introduce Grover's algorithm and how it can be used to solve unstructured search problems. This algorithm can speed up an unstructured search problem quadratically.

Suppose you are given a large list of N items. Among these items there is one item with a unique property that we wish to locate; we will call this one the winner w. 

To find the the marked item using classical computation, one would have to check on average N/2 of these boxes, and in the worst case, all N
 of them. On a quantum computer, however, we can find the marked item in roughly 
√N steps with Grover's algorithm.

Before looking at the list of items, we have no idea where the marked item is. Therefore, any guess of its location is as good as any other, which can be expressed in terms of a uniform superposition: 

![eq1](assets/equations/eq1.png)

If at this point we were to measure in the standard basis 
{|x⟩}, this superposition would collapse, according to the fifth quantum law, to any one of the basis states with the same probability of ![eq2](assets/equations/eq2.png).

Our chances of guessing the right value w is therefore 1 in 2^n, as could be expected. Hence, on average we would need to try about N/2 = 2^(n-1) times to guess the correct item.
This algorithm has a nice geometrical interpretation in terms of two reflections, which generate a rotation in a two-dimensional plane. The only two special states we need to consider are the winner |w⟩ and the uniform superposition |s⟩. These two vectors span a two-dimensional plane in the vector space ![eq3](assets/equations/eq3.png).

They are not quite perpendicular because |w⟩ occurs in the superposition with amplitude ![eq4](assets/equations/eq4.png) as well. We can, however, introduce an additional state |s′⟩ that is in the span of these two vectors, which is perpendicular to |w⟩ and is obtained from |s⟩ by removing |w⟩ and rescaling.

Step 1: The amplitude amplification procedure starts out in the uniform superposition |s⟩, which is easily constructed from: ![eq5](assets/equations/eq5.png)

![grover-step1](assets/diagrams/grover-step1.png)

The left graphic corresponds to the two-dimensional plane spanned by perpendicular vectors|w⟩ and |s′⟩ which allows to express the initial state as: 

![eq7](assets/equations/eq7.png)

The right graphic is a bar graph of the amplitudes of the state |s⟩.

Step 2: We apply the oracle reflection ![eq8](assets/equations/eq8.png) to the state |s⟩:

![grover-step2](assets/diagrams/grover-step2.png)

Geometrically this corresponds to a reflection of the state |s⟩ about |s′⟩. This transformation means that the amplitude in front of the |w⟩ state becomes negative, which in turn means that the average amplitude (indicated by a dashed line) has been lowered.

Step 3: We now apply an additional reflection ![eq9](assets/equations/eq9.png) about the state ![eqA](assets/equations/eqA.png): ![eqB](assets/equations/eqB.png). This transformation maps the state to ![eqC](assets/equations/eqC.png) and completes the transformation.

![grover-step3](assets/diagrams/grover-step3.png)

Two reflections always correspond to a rotation. The transformation UsUf
rotates the initial state |s⟩ closer towards the winner |w⟩. The action of the reflection 
Us in the amplitude bar diagram can be understood as a reflection about the average amplitude. Since the average amplitude has been lowered by the first reflection, this transformation boosts the negative amplitude of |w⟩ to roughly three times its original value, while it decreases the other amplitudes. We then go to step 2 to repeat the application. This procedure will be repeated several times to zero in on the winner.

After t steps we will be in the state |ψt⟩  where: 

![grover-search](assets/diagrams/grover-search.jpeg)

> The three stages of the 3-qubit Grover search algorithm: initialization, oracle, and amplification. Credit: C. Figgatt et al. Published in Nature Communications
- Source: [phys.org](https://scx2.b-cdn.net/gfx/news/2018/groversearch.jpg)

#### Quantum Fourier Transform

The Fourier transform occurs in many different versions throughout classical computing, in areas ranging from signal processing to data compression to complexity theory. The quantum Fourier transform (QFT) is the quantum implementation of the discrete Fourier transform over the amplitudes of a wavefunction.


The discrete Fourier transform acts on a vector (x0,...,xN−1) and maps it to the vector (y0,...,yN−1) according to the formula:



where 

Similarly, the quantum Fourier transform acts on a quantum state and maps it to the quantum state according to the formula:



We see that only the amplitudes of the state were affected by this transformation.

This can also be expressed as the unitary matrix:



The quantum Fourier transform (QFT) transforms between two bases, the computational (Z) basis, and the Fourier basis. The H-gate is the single-qubit QFT, and it transforms between the Z-basis states |0⟩ and |1⟩ to the X-basis states|+⟩
 and |−⟩. In the same way, all multi-qubit states in the computational basis have corresponding states in the Fourier basis. The QFT is simply the function that transforms between these bases.



(We note states in the Fourier basis using the tilde (~)).

Counting in the Fourier basis:

In the computational basis, we store numbers in binary using the states 
|0⟩ and |1⟩:






The frequency with which the different qubits change; the leftmost qubit flips with every increment in the number, the next with every 2 increments, the third with every 4 increments, and so on. In the Fourier basis, we store numbers using different rotations around the Z-axis:



The number we want to store dictates the angle at which each qubit is rotated around the Z-axis. In the state |0~⟩, all qubits are in the state |+⟩. As seen in the example above, to encode the state |5~⟩ on 4 qubits, we rotated the leftmost qubit by  full turns ( radians). The next qubit is turned double this radians, or 10/16 full turns, this angle is then doubled for the qubit after, and so on.

Again, note the frequency with which each qubit changes. The leftmost qubit (qubit 0) in this case has the lowest frequency, and the rightmost the highest.



### Uncertainty Model

We now construct a circuit that loads the uncertainty model. This can be achieved by creating a quantum state in a register of $n_z$ qubits that represents $Z$ following a standard normal distribution. This state is then used to control single qubit Y-rotations on a second qubit register of $K$ qubits, where a $|1\rangle$ state of qubit $k$ represents the default event of asset $k$.

The resulting quantum state can be written as

$ |\Psi\rangle = \sum_{i=0}^{2^{n_z}-1} \sqrt{p_z^i} |z_i \rangle \bigotimes_{k=1}^K
\left( \sqrt{1 - p_k(z_i)}|0\rangle + \sqrt{p_k(z_i)}|1\rangle\right) $

where we denote by $z_i$ the $i$-th value of the discretized and truncated $Z$ [Egger2019].

In [ ]:
from qiskit_finance.circuit.library import GaussianConditionalIndependenceModel as GCI

u = GCI(n_z, z_max, p_zeros, rhos)
u.draw()

We now use the simulator to validate the circuit that constructs $|\Psi\rangle$ and compute the corresponding exact values for: 

- Expected loss $$ 
- PDF and CDF of $L$ // <--- what is this???
- value at risk $VaR(L)$ and corresponding probability
- conditional value at risk $CVaR(L)$ 

In [ ]:
# run the circuit and analyze the results
job = execute(u, backend=Aer.get_backend("statevector_simulator"))

# analyze uncertainty circuit and determine exact solutions
p_z = np.zeros(2 ** n_z)
p_default = np.zeros(K)
values = []
probabilities = []
num_qubits = u.num_qubits
state = job.result().get_statevector()
if not isinstance(state, np.ndarray):
    state = state.data
for i, a in enumerate(state):

    # get binary representation
    b = ("{0:0%sb}" % num_qubits).format(i)
    prob = np.abs(a) ** 2

    # extract value of Z and corresponding probability
    i_normal = int(b[-n_z:], 2)
    p_z[i_normal] += prob

    # determine overall default probability for k
    loss = 0
    for k in range(K):
        if b[K - k - 1] == "1":
            p_default[k] += prob
            loss += lgd[k]
    values += [loss]
    probabilities += [prob]

values = np.array(values)
probabilities = np.array(probabilities)

expected_loss = np.dot(values, probabilities)

losses = np.sort(np.unique(values))
pdf = np.zeros(len(losses))
for i, v in enumerate(losses):
    pdf[i] += sum(probabilities[values == v])
cdf = np.cumsum(pdf)

i_var = np.argmax(cdf >= 1 - alpha)
exact_var = losses[i_var]
exact_cvar = np.dot(pdf[(i_var + 1) :], losses[(i_var + 1) :]) / sum(pdf[(i_var + 1) :])

In [ ]:
print("Expected Loss E[L]:                %.4f" % expected_loss)
print("Value at Risk VaR[L]:              %.4f" % exact_var)
print("P[L <= VaR[L]]:                    %.4f" % cdf[exact_var])
print("Conditional Value at Risk CVaR[L]: %.4f" % exact_cvar)

In [ ]:
# plot loss PDF, expected loss, var, and cvar
plt.bar(losses, pdf)
plt.axvline(expected_loss, color="green", linestyle="--", label="E[L]")
plt.axvline(exact_var, color="orange", linestyle="--", label="VaR(L)")
plt.axvline(exact_cvar, color="red", linestyle="--", label="CVaR(L)")
plt.legend(fontsize=15)
plt.xlabel("Loss L ($)", size=15)
plt.ylabel("probability (%)", size=15)
plt.title("Loss Distribution", size=20)
plt.xticks(size=15)
plt.yticks(size=15)
plt.show()

In [ ]:
# plot results for Z
plt.plot(z_values, p_z, "o-", linewidth=3, markersize=8)
plt.grid()
plt.xlabel("Z value", size=15)
plt.ylabel("probability (%)", size=15)
plt.title("Z Distribution", size=20)
plt.xticks(size=15)
plt.yticks(size=15)
plt.show()

In [ ]:
# plot results for default probabilities
plt.bar(range(K), p_default)
plt.xlabel("Asset", size=15)
plt.ylabel("probability (%)", size=15)
plt.title("Individual Default Probabilities", size=20)
plt.xticks(range(K), size=15)
plt.yticks(size=15)
plt.grid()
plt.show()

### Expected Loss

To estimate the expected loss, we first apply a weighted sum operator to sum up individual losses to total loss:

$ \mathcal{S}: |x_1, ..., x_K \rangle_K |0\rangle_{n_S} \mapsto |x_1, ..., x_K \rangle_K |\lambda_1x_1 + ... + \lambda_K x_K\rangle_{n_S} $

The required number of qubits to represent the result is given by

$ n_s = \lfloor \log_2( \lambda_1 + ... + \lambda_K ) \rfloor + 1 $

Once we have the total loss distribution in a quantum register, we can use the techniques described in [Woerner2019] to map a total loss $ L \in \{0, ..., 2^{n_s}-1\} $ to the amplitude of an objective qubit by an operator

$ | L \rangle_{n_s}|0\rangle \mapsto
| L \rangle_{n_s} \left( \sqrt{1 - L/(2^{n_s}-1)}|0\rangle + \sqrt{L/(2^{n_s}-1)}|1\rangle \right) $

which allows to run amplitude estimation to evaluate the expected loss.


In [ ]:
from qiskit.circuit.library import WeightedAdder, LinearAmplitudeFunction

# add Z qubits with weight/loss 0
agg = WeightedAdder(n_z + K, [0] * n_z + lgd)
# define linear objective function
breakpoints = [0]
slopes = [1]
offsets = [0]
f_min = 0
f_max = sum(lgd)
c_approx = 0.25

objective = LinearAmplitudeFunction(
    agg.num_sum_qubits,
    slope=slopes,
    offset=offsets,
    # max value that can be reached by the qubit register (will not always be reached)
    domain=(0, 2 ** agg.num_sum_qubits - 1),
    image=(f_min, f_max),
    rescaling_factor=c_approx,
    breakpoints=breakpoints,
)

Create the state preparation circuit:

In [ ]:
# define the registers for convenience and readability
qr_state = QuantumRegister(u.num_qubits, "state")
qr_sum = QuantumRegister(agg.num_sum_qubits, "sum")
qr_carry = QuantumRegister(agg.num_carry_qubits, "carry")
qr_obj = QuantumRegister(1, "objective")

# define the circuit
state_preparation = QuantumCircuit(qr_state, qr_obj, qr_sum, qr_carry, name="A")

# load the random variable
state_preparation.append(u.to_gate(), qr_state)

# aggregate
state_preparation.append(agg.to_gate(), qr_state[:] + qr_sum[:] + qr_carry[:])

# linear objective function
state_preparation.append(objective.to_gate(), qr_sum[:] + qr_obj[:])

# uncompute aggregation
state_preparation.append(agg.to_gate().inverse(), qr_state[:] + qr_sum[:] + qr_carry[:])

# draw the circuit
state_preparation.draw()

Before we use QAE to estimate the expected loss, we validate the quantum circuit representing the objective function by just simulating it directly and analyzing the probability of the objective qubit being in the $|1\rangle$ state, i.e., the value QAE will eventually approximate.

In [ ]:
job = execute(state_preparation, backend=Aer.get_backend("statevector_simulator"))
# evaluate resulting statevector
value = 0
state = job.result().get_statevector()
if not isinstance(state, np.ndarray):
    state = state.data
for i, a in enumerate(state):
    b = ("{0:0%sb}" % (len(qr_state) + 1)).format(i)[-(len(qr_state) + 1) :]
    prob = np.abs(a) ** 2
    if prob > 1e-6 and b[0] == "1":
        value += prob

print("Exact Expected Loss:   %.4f" % expected_loss)
print("Exact Operator Value:  %.4f" % value)
print("Mapped Operator value: %.4f" % objective.post_processing(value))

Next we run QAE to estimate the expected loss with a quadratic speed-up over classical Monte Carlo simulation.

In [ ]:
# set target precision and confidence level
epsilon = 0.01
alpha = 0.05

qi = QuantumInstance(Aer.get_backend("aer_simulator"), shots=100)
problem = EstimationProblem(
    state_preparation=state_preparation,
    objective_qubits=[len(qr_state)],
    post_processing=objective.post_processing,
)
# construct amplitude estimation
ae = IterativeAmplitudeEstimation(epsilon, alpha=alpha, quantum_instance=qi)
result = ae.estimate(problem)

# print results
conf_int = np.array(result.confidence_interval_processed)
print("Exact value:    \t%.4f" % expected_loss)
print("Estimated value:\t%.4f" % result.estimation_processed)
print("Confidence interval: \t[%.4f, %.4f]" % tuple(conf_int))

### Cumulative Distribution Function

Instead of the expected loss (which could also be estimated efficiently using classical techniques) we now estimate the cumulative distribution function (CDF) of the loss. Classically, this either involves evaluating all the possible combinations of defaulting assets, or many classical samples in a Monte Carlo simulation. Algorithms based on QAE have the potential to significantly speed up this analysis in the future.

To estimate the CDF, i.e., the probability $\mathbb{P}[L \leq x]$, we again apply $\mathcal{S}$ to compute the total loss, and then apply a comparator that for a given value $x$ acts as

$$\begin{split} \mathcal{C}: |L\rangle_n|0> \mapsto
\begin{cases}
|L\rangle_n|1> & \text{if}\quad L \leq x \\
|L\rangle_n|0> & \text{if}\quad L > x
\end{cases}\end{split}$$

The resulting quantum state can be written as

$ \sum_{L = 0}^{x} \sqrt{p_{L}}|L\rangle_{n_s}|1\rangle +
\sum_{L = x+1}^{2^{n_s}-1} \sqrt{p_{L}}|L\rangle_{n_s}|0\rangle $

Where we directly assume the summed up loss values and corresponding probabilities instead of presenting the details of the uncertainty model.

The CDF $(x)$ equals the probability of measuring $|1\rangle$ in the objective qubit and QAE can be directly used to estimate it.

In [ ]:
# set x value to estimate the CDF
x_eval = 2

comparator = IntegerComparator(agg.num_sum_qubits, x_eval + 1, geq=False)
comparator.draw()

In [ ]:
def get_cdf_circuit(x_eval):
    # define the registers for convenience and readability
    qr_state = QuantumRegister(u.num_qubits, "state")
    qr_sum = QuantumRegister(agg.num_sum_qubits, "sum")
    qr_carry = QuantumRegister(agg.num_carry_qubits, "carry")
    qr_obj = QuantumRegister(1, "objective")
    qr_compare = QuantumRegister(1, "compare")

    # define the circuit
    state_preparation = QuantumCircuit(qr_state, qr_obj, qr_sum, qr_carry, name="A")

    # load the random variable
    state_preparation.append(u, qr_state)

    # aggregate
    state_preparation.append(agg, qr_state[:] + qr_sum[:] + qr_carry[:])

    # comparator objective function
    comparator = IntegerComparator(agg.num_sum_qubits, x_eval + 1, geq=False)
    state_preparation.append(comparator, qr_sum[:] + qr_obj[:] + qr_carry[:])

    # uncompute aggregation
    state_preparation.append(agg.inverse(), qr_state[:] + qr_sum[:] + qr_carry[:])

    return state_preparation


state_preparation = get_cdf_circuit(x_eval)

Again, we first use quantum simulation to validate the quantum circuit.

In [ ]:
job = execute(state_preparation, backend=Aer.get_backend("statevector_simulator"))
state_preparation.draw()

In [ ]:
# evaluate resulting statevector
var_prob = 0
state = job.result().get_statevector()
if not isinstance(state, np.ndarray):
    state = state.data
for i, a in enumerate(state):
    b = ("{0:0%sb}" % (len(qr_state) + 1)).format(i)[-(len(qr_state) + 1) :]
    prob = np.abs(a) ** 2
    if prob > 1e-6 and b[0] == "1":
        var_prob += prob
print("Operator CDF(%s)" % x_eval + " = %.4f" % var_prob)
print("Exact    CDF(%s)" % x_eval + " = %.4f" % cdf[x_eval])

Next we run QAE to estimate the CDF for a given $x$.

In [ ]:
# set target precision and confidence level
epsilon = 0.01
alpha = 0.05

qi = QuantumInstance(Aer.get_backend("aer_simulator"), shots=100)
problem = EstimationProblem(state_preparation=state_preparation, objective_qubits=[len(qr_state)])
# construct amplitude estimation
ae_cdf = IterativeAmplitudeEstimation(epsilon, alpha=alpha, quantum_instance=qi)
result_cdf = ae_cdf.estimate(problem)

# print results
conf_int = np.array(result_cdf.confidence_interval)
print("Exact value:    \t%.4f" % cdf[x_eval])
print("Estimated value:\t%.4f" % result_cdf.estimation)
print("Confidence interval: \t[%.4f, %.4f]" % tuple(conf_int))

## Results


We went on to show how our algorithm can be applied to the task of pricing an asset using real quantum hardware. We chose the simple model of a treasury bill whose value depends only on the interest rate. This simple experiment on 5 qubits confirmed the theoretical convergence rate of our algorithm and underlines its potential advantage compared to classical Monte Carlo methods.

In order to further demonstrate the capabilities of our algorithm, we used classical simulations of quantum hardware to show that it can also be applied to speed up the computation of risk measures of a simple two-asset portfolio. Nevertheless, we notice that to achieve quantum advantage in a real-world scenario, the quality of current quantum hardware needs to be improved. Errors arising from the limited coherence time and cross-talk when measuring the states of qubits need to be substantially suppressed. Furthermore, the number of qubits must be increased.

However, the current pace of advances in research directed at improving both quantum hardware and quantum algorithms makes us optimistic that real quantum advantage in risk analysis can be achieved in the future.

## Conclusion

By now, most people have heard that quantum computing is a revolutionary technology that leverages the bizarre characteristics of quantum mechanics to solve certain problems faster than regular computers can. Those problems range from the worlds of mathematics to retail business, and physics to finance.

But, as we have seen this year, Quantum computers are exceedingly difficult to engineer, build and program. As a result, they are crippled by errors in the form of noise, faults and loss of quantum coherence.

While classical computers are also affected by various sources of errors, these errors can be corrected with a modest amount of extra storage and logic. Quantum error­ correction schemes do exist but consume such a large number of qubits (quantum bits) that relatively few qubits remain for actual computation. So when only a couple of qubits are required it could be feasible, but when the number required increases the computing is unviable.   That reduces the size of the computing task to a tiny fraction of what could run on defect-­free hardware.

So, all in all, the application of this quantum algorithm (or quantum computing in general) is not feasible due to the noise and the quantity of qubits required.

## References

1. [Purpose of Credit Risk Analysis](https://corporatefinanceinstitute.com/resources/knowledge/credit/purpose-of-credit-risk-analysis/)
2. [How Quantum Computing Could Change Financial Services](https://www.mckinsey.com/industries/financial-services/our-insights/how-quantum-computing-could-change-financial-services)


Quantum Risk Analysis. Stefan Woerner, Daniel J. Egger. [Woerner2019]
- https://www.nature.com/articles/s41534-019-0130-6

Credit Risk Analysis using Quantum Computers. Egger et al. (2019) [Egger2019]
- https://arxiv.org/abs/1907.03044

Quantum Amplitude Amplification and Estimation. Gilles Brassard et al.
- http://arxiv.org/abs/quant-ph/0005055